In [1]:
import torch
import torch.nn as nn
import click
import asm2vec
import logging
from util.log_helper import get_logger
logger = get_logger("test.log")

def cosine_similarity(v1, v2):
    return (v1 @ v2 / (v1.norm() * v2.norm())).item()

【2022-06-23 22:14:00】INFO: Log File Save To:     /home/yhk/github/asm2vec-pytorch/log/test.log


In [2]:
mpath = "./model.pt"
ipath1 = "./test_sample/binutils-2.37_x86-64_O0_clang_addr2line_sym.__libc_csu_init.asm"
ipath2 = "./test_sample/binutils-2.37_x86-64_O0_clang_addr2line_sym.__libc_csu_init.asm"
device = "auto"
epochs = 10
lr = 0.025

In [3]:
import os
file_l = os.listdir(ipath)

NameError: name 'ipath' is not defined

In [ ]:
count = 0
for file in file_l:
    if "_O3_" in file:
        os.system("mv ./radare2_export_asm_task7/{} ./test_sample/".format(file))
        count+=1
    if count==900:
        break

In [5]:
ipath = "./radare2_export_asm_task7/"

functions, tokens_new = asm2vec.utils.load_data(ipath)

348412it [12:19, 471.38it/s] 


In [6]:
len(functions)

348412

In [7]:
if device == 'auto':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load model, tokens
model, tokens = asm2vec.utils.load_model(mpath, device=device)
print(model.embeddings_f)

tokens.update(tokens_new)
model.update(len(functions), tokens.size())
model = model.to(device)


Embedding(347099, 200)


In [8]:
logging.info("Start train...")
# train function embedding
model = asm2vec.utils.train(
    functions,
    tokens,
    model=model,
    epochs=epochs,
    device=device,
    mode='test',
    learning_rate=lr
)
logging.info("Completed train!")

【2022-06-23 22:27:04】INFO: Start train...
35168it [11:57, 49.04it/s]
35168it [13:07, 44.66it/s]
35168it [12:56, 45.27it/s]
35168it [11:32, 50.81it/s]
35168it [11:32, 50.81it/s]
35168it [11:32, 50.77it/s]
35168it [11:25, 51.32it/s]
35168it [11:42, 50.08it/s]
35168it [10:44, 54.58it/s]
35168it [12:20, 47.49it/s]
【2022-06-24 00:39:09】INFO: Completed train!


In [9]:
for i,func_name in enumerate(func_names):
    if "zlib_advance" in func_name:
        print(func_name)

NameError: name 'func_names' is not defined

In [9]:
func_name_l = [function.meta["name"] for function in functions]

In [10]:
embedding_l = model.to("cpu").embeddings_f.weight.tolist()

In [3]:
import hnswlib
import numpy as np
import pickle

def top_k_vector_find_hnsw(query_embedding,embedding_library,k=5):
    dim = len(embedding_library[0])
    num_elements = len(embedding_library)
    
    ids = np.arange(num_elements)
    
    # Declaring index
    p = hnswlib.Index(space = 'cosine', dim = dim) # possible options are l2, cosine or ip

    # Initializing index - the maximum number of elements should be known beforehand
    p.init_index(max_elements = num_elements, ef_construction = 200, M = 16)
    
    # Element insertion (can be called several times):
    p.add_items(embedding_library, ids)
    
    # Controlling the recall by setting ef:
    p.set_ef(50) # ef should always be > k

    # Query dataset, k - number of closest elements (returns 2 numpy arrays)
    labels, distances = p.knn_query(query_embedding, k = k)
    return labels,distances
    
def get_top_k_sim_func(query_embedding,embedding_library,embedding_library_func_names,k=5):
    """
        
    """
    top_k_funcs = []
    top_k_indexs,sim_scores = top_k_vector_find_hnsw(query_embedding,embedding_library,k)
#     for i,top_k_index in enumerate(top_k_indexs):
#         top_k_funcs.append([embedding_library_func_names[j] for j in top_k_indexs])
#     return top_k_funcs,sim_scores,top_k_funcs_indexs
    return sim_scores,top_k_funcs_indexs

In [4]:
sim_scores,top_k_funcs_indexs = get_top_k_sim_func(embedding_l,embedding_l,func_name_l)

NameError: name 'embedding_l' is not defined

In [1]:
sim_scores

NameError: name 'sim_scores' is not defined